In [104]:
# !pip install octis
# !pip install contextualized-topic-models==2.3.0
# !pip install datasets
# !pip install pyldavis

In [105]:
import nltk
import pandas as pd
from nltk.corpus import stopwords as stop_words
from datasets import load_dataset
from octis.dataset.dataset import Dataset
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import string
from gensim.utils import deaccent
import warnings
from sentence_transformers import SentenceTransformer
import scipy.sparse
from contextualized_topic_models.datasets.dataset import CTMDataset
from sklearn.preprocessing import OneHotEncoder
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.models.ctm import ZeroShotTM
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import common_texts
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO

In [106]:
is_combined = True
perc = 0.10 # 10% of dataset

nltk.download('stopwords')

language = 'en' if is_combined else 'all_languages'
stopwords = list(stop_words.words('english')) if is_combined else list(stop_words.words(stop_words.fileids())) # from every language
embedding_model = 'paraphrase-distilroberta-base-v2' if is_combined else 'paraphrase-multilingual-mpnet-base-v2'

num_epochs_ctm = 20
num_topics = 50
num_topics_word = 700

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [107]:
dataset = load_dataset('amazon_reviews_multi', language)

In [108]:
def data(part):
  data = dataset[part].to_pandas()
  review_body = list(data['review_body'])
  return data, review_body

train, review_body = data('train')
test, review_body_test = data('test')
validation, review_body_validation = data('validation')

def make_review_body(data, perc):
  arr = sklearn.utils.shuffle(np.arange(len(data)), random_state=42)[0:int(len(data) * perc)]
  temp = data.iloc[arr].reset_index(drop=True)
  review_body = list(temp['review_body'])
  temp = temp.reset_index()
  temp = temp.rename(columns = {'index': 'message_id', 'review_body': 'message'})
  return temp, review_body

temp_train, review_body = make_review_body(train, perc)
temp_test, test_review_body = make_review_body(test, 1)
temp_validation, validation_review_body = make_review_body(validation, 1)

In [109]:
documents = [line.strip() for line in review_body + test_review_body]
test_documents = [line.strip() for line in test_review_body]

In [110]:
from octis.models.LDA import LDA
from octis.models.CTM import CTM
from octis.models.ETM import ETM
from octis.models.ProdLDA import ProdLDA

prodlda_args = {
  'num_epochs': 20,
  'num_layers': 2,
  'num_neurons': 100,
  'num_topics': 50,
}

model = ProdLDA(num_topics=num_topics, num_epochs=prodlda_args['num_epochs'],
        num_layers=prodlda_args['num_layers'], num_neurons=prodlda_args['num_neurons'],
        use_partitions=False)

In [111]:
octis_df = pd.DataFrame(train['review_body'])
octis_df['train'] = 'train'

test_octis_df = pd.DataFrame(test['review_body'])
test_octis_df['train'] = 'test'

corpus = pd.concat([octis_df, test_octis_df]).reset_index(drop=True)

corpus = corpus[0:2000] # just for now

vocabulary = list(set(' '.join([i for i in corpus['review_body']]).split()))

corpus.to_csv('/content/corpus.tsv', sep="\t")

In [119]:
f = open('/content/vocabulary.txt','w')
for word in vocabulary:
	f.write(word+"\n")
f.close()

In [122]:
import os
import string
from octis.preprocessing.preprocessing import Preprocessing
os.chdir(os.path.pardir)

# Initialize preprocessing
preprocessor = Preprocessing(vocabulary=None, max_features=None,
                             remove_punctuation=True, punctuation=string.punctuation,
                             lemmatize=True, stopword_list='english',
                             min_chars=1, min_words_docs=0)
# preprocess
dataset = preprocessor.preprocess_dataset(documents_path=r'/content/corpus.tsv')

# save the preprocessed dataset
dataset.save('hello_dataset')

100%|██████████| 2001/2001 [00:32<00:00, 62.20it/s]


created vocab
4631


In [116]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder("/content")

In [120]:
trained_model = model.train_model(dataset)

ValueError: ignored